In [40]:
using GR
using Random
import Dates
using BenchmarkTools

In [41]:
#### specify path of Julia Code for 2D Models ####
path = "/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"


##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
op_model = "AV_IN_Model"
#op_model = "IN_Costfunction"

#op_model = "Intrusion_Force"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path);

In [42]:
#### parameters of the agents ####
δt = 0.01
p = [1.6, 1.2, 3.0, 1.0, 0.3, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.3, 0.6283185307179586]
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, ζ_h, ζ_v, r_priv, ϕ"

### of the system ###
N = 2
system_size = (5.0, 5.0) #important for periodic boundaries
#system_size = (35.0, 35.0) #important for periodic boundaries

### of the simulation ###
sim_p = [0.0, 3.0, 0.04, δt, p[1]*p[2]] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = Create_Geometry_Bottleneck(0.2, 0.8, 0.1, system_size)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

β, D, x_min, y_min, ϵ = π/2, sqrt(10), 0.0, 0.0, 0.001
Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

#D, x_min, y_min, ϵ = 3.0, 0.5, 1.5, 0.2
#Init_Three_Agents!(menge, D, x_min, y_min, ϵ)

v2, v1 = 0.5, 1.5
T2, T1 = 1.0, 0.7
D, ϵ = 3.0, 0.0
x_min, y_min = 2.5, 2.5

#Init_Overtaking!(menge, D, x_min, y_min, ϵ, v1, v2, T1, T2);

### Sparse Bottleneck ###

#Init_Agent_Bottleneck!(1.9*pi, 4, menge, system_size)

#ϕ1, ϕ2, dist1, dist2 = 2.5*π/2, 3.5*π/2, 4.0, 4.0
#Init_2_Agents_Bottleneck!(ϕ1, ϕ2, dist1, dist2, menge, system_size)

#ϕ1, ϕ2, ϕ3, dist1, dist2, dist3 = 0.0, 3*π/2, 3.2*π/2, 4.0, 4.0, 3.5
#Init_3_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, dist1, dist2, dist3, menge, system_size)


#ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4 = 1.2*π, π, 3.2*π/2, 0.0, 4.0, 4.0, 3.8, 4.0
#Init_4_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4, menge, system_size)

In [43]:
#@benchmark begin
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
#end

In [44]:
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 6, -1, 6)
        GR.setviewport(0.0, 1, 0.0, 1)
    
        #plot_goal(goal1)
        #plot_goal(goal2)

        #plot_geometry(geometry_x, geometry_y, 3.6, 20)
        #plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        
        plot_agents(positions[j, :], headings[j, :], 3.6, 100, 1.2, 999, 0.1)
                
        model = "AV*IN-model"
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.8, 0.95, timer)
        GR.text(0.8, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [45]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQABNiptZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAANiWWIhAAa//731LfMsu1Tb7YLqPeiniZfQ3UlAZuWxO06gAAAAwAAAwAAAwDpVIJjfO4A0Yy0AAAW43cqxW9GcwA/xIpoH0oiK0e6lVH4r43cNeuPzJijpQ8PVmBUHc/27v0J6vi2PmzIdBU6JN1XJZWIwH5bQyPdbzj01ocpcVhkHLxEJ6XUBK3ZBWNnsjjFzCXSpZg+aRVirLdKQim/Y9sjZw+RwU0ASkrwcOWTYw2/+e7XcVXrmhN5d0UbCxt8bejthpfjM0i8hv0q69aWnjEqyCmw++dH0+jdHkBbYCXZtqLgoZtywOYErIqkQTxTmoEkHItDxHDrTtlx+AsvM2p3faNFgngQnqqs5H+BWVIJvxaifcKiTmKEQ6xPWVTNhD6DgN9PnO/uYmnXq0zXYb3qctCsAvAsCu92xXBgCVk/R2cxJzkhvMg3QEYy6pI1/DRa/ZtsivvOMLN2PbwIjPmiux0O3uyGL/+R9m08x2omBwZ6NlVd2JIIyc2+i3V8DGPEmodA6Nb7BqKqa6WtsKvH10h0FXBpMHfROCptscMl0CvTYyWqtFlri+JE7tRz5HYcOqet/NK1hOeSjQgngbabd13hesxkuqvEN1zP8eR2o1Y+3lDzej7vpzv6dJgx5NBs/Sr+Xaf4w4uHKOdj3Mi063kgWvemlJP0NsErOn1igwikOtNjDC0gsroQbWIgPmlAAwuF/lB2GGyieHUJIXGQ/S5LNnnnvmZggzSj33Kk8LlCUi/PgV2OKQDxxzLwRmRZa/VCSNemYuDAfcQ2HwtQBtdC0i++KZFLHkojwO74elzH+QA9q+jm3sK4zEB2FFmIjpryCSbESzIrPrUOuT/AX9NbZRUXprj2SQBT3WCfmKlHDZ49ons+4lBBf/bfl9Z6oigHsILAkn26u/k1DFg9bMKwe309HuBoDPwmyLpWCaqHcLW+QYHy9AidOJLq7ualtugtMZ+AWJSOv0buwVrRng8NMKOJhowCkfznVtZbHusb2VNiDmfoa11wURgbE+P6J6z63GNwBflO7aulrvA2WGZ4sh9bhirdcqO1hfzQf5nLOfLZrg6sMUoyJgaAAMGpJvK/5NDDdx2vLpEgFj9ow9uGfOI+uHGkkq43eKYBNdvSt5j/+rz1UP5bkvHyGlO9hbcP8p4/b738cNXFXQEB5K+u517v3UDbAZvY7nze7DtvrGecHPiH8lxyVrZrpZtChsxgEa+dbL/fjmkw+M62tM5cBzMKnaXaSia3GVF03zTfXOb4FR5Mo31yO6i9mULEDVrUYG4uUMlRu0ks7LSGm7uh4Qg1drxqniy2G4m7ZWfZ4h0GRxO7al1u1Btnq2SFtO7DtLIy1axmJ3gh1DI6P2/EaEVBQC+9YIqT5QluluWqYGWulSEsAL8s8R2dl/LHYgSU/voV79RaAGFiOkzlnSauBC11lqCFkHMx3AIng9uQeSX0gVFy1etqyv++S4UYaRDkU/tAyW1z/MU7s1vbX92LdptwtViM84vCO3PqbA9eR1RvqReDEr9F03yEO81ie5P0Hq4hNrMK6mKM27r7zaYdYDN0R+ij/tM5JR7fWw6hXywZ+RTCwNE1aX1fFZb7oMDnVkblbNCCAWlGHxXTCP0FuUlSmmbveLp8pw5U2dYvs2dotlo6/3C7FqSD77QIDdW/ll/z6vHeTyGOF5nY14lDAnh8XZEl91ytVi8D0ndLmxaGiEUJqWvw4y+5XExi21RBwHlXDH4EsuTxeWGJp6XaOCO7eDJPcmVAY4p22IpWQRFpWk8wEtgg2dBmvKn4FC3GAg58J35zcPGdRq27gGRaphfCX/B68qYkSAn9XFhA9zgHQxFL0XTuSwF3Fsq0JbvkfaS/dSfF2Ot9YDWMO+V1Tcp6mI4F2SnRSIN6kttKntmrX+v+icWl5m9MitpSCjYH6UkbPdhchpoofXaOApBIvsF5OEew9y1vU1yQ1SQ1HAqtSm94iK9B2sSs7O02UjyG+n/nzTdi05BpYtDG0MgvoXw3VnDJfFI4dpHuqJ+SJILxvWTt6cZ1OHJvWFdiDNebYgukEc4Lw6nZjU/XVNCkyEmMaKJ94v2QHl9LVnXYt2sXSIQgRPHAqr7i4XUHYhhFBRS+7w8UJdbvlys+HK7Gau6RXFKQnwxFE9dPhEhHSnS8C56abKxycwTs5uyoCUDYgpXgWumMF6suyIj66BPQZFrR1o7Iyps4wKBC1XMwrjR3fx36ZNUynmLZFhmJjhiOgkP8UA+NRvWiwcR4zepGs1P9WuPDWCiv+b36aHaUc50ni1E9gyQkG1Nb8XRrLyvy+MxhR9II02TWWbDQXfcucy8GvNw+yXTmQN+H6NT//bFYnH8OtetA3q+eOLWf62fm04J/R6Sb9/ydXVeAExXucXd6gYaNHQEhclorZ5BuIsKINXrNYdKAKWQY4cEAOk/aECXbrhX3vyVrSkR7LnjHVREDNmeqnPZDnPp1kzEO9J/8H8KMCmSuIodBI/FxeABl/E0DEoXs7qurAcAMi79CeKe7jvxl+co2x2pCjCPeGTPxfOuEPVfCseBRS4vSOqSOBapvpmOextLOVElS7jizVxuHUqz9CnAGGOGNX6XxiAVhrS6jEeOfoGLoA1mxN/bK0IX/3AoUfE8SMDQJj8ZTYgRw7KSF0KdZ5nvxeTaK47g0CImtkv+Z8Nq/zQJ6hna0SELIq/cTIqbfkvhqzSaPOop8D7DFJxg/RUmjYlY8zTc67feK6XW5ltrSS2fzuLJdW3lbm1Brqmwpu2VmwEM7lyqKkcqS0akhickRiujfR7DaBdInpxLl04vqEQw7d1odtCITS5VeYtAeyrdh2IcOQa3lFWhoyn5p/T5bH2PIhWfDHV2Hf69O2m0UlFGzwn/uJl5+unAPNs5wqEi9J55Y51Vcf3jm7f//sqFZpG1ete3/Xrn

In [17]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "90_CollisionFreeSpeed"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

Created Directory /90_CollisionFreeSpeed.


In [ ]:
samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [11]:
name = "Agents_Test"
path = "/home/jakob/Dokumente/Renne_Software/UMANS/examples/agents/"

Write_Agents_XML(menge, name, path)

298